<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Timeliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [37]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import glob
import datetime

In [18]:
!pip install geopandas
import geopandas as gpd

# Data Preparation

## Download MBTA Data

In [19]:
if not os.path.exists("data"):
  !wget https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
if not os.path.exists("MBTA Bus Arrival Departure Aug-Sept 2018.csv"):
  !unzip data

if not os.path.exists("data.1"):
  !wget https://www.arcgis.com/sharing/rest/content/items/1bd340b39942438685d8dcdfe3f26d1a/data
if not os.path.exists("MBTA Bus Arrival Departure Apr-June 2019.csv"):
  !unzip data.1

## Load MBTA Arrival Data

In [20]:
df2 = pd.read_csv("MBTA Bus Arrival Departure Apr-June 2019.csv")
df2

,service_date,route_id,direction,half_trip_id,stop_id,stop_name,stop_sequence,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2019-04-01,01,Inbound,42976988.0,75,mit,4.0,Midpoint,Schedule,1900-01-01 05:19:00,1900-01-01 05:21:20,NaN,NaN
1,2019-04-01,01,Inbound,42976988.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01 05:23:00,1900-01-01 05:24:17,NaN,NaN
2,2019-04-01,01,Inbound,42976988.0,187,masta,6.0,Midpoint,Schedule,1900-01-01 05:25:00,1900-01-01 05:26:05,NaN,NaN
3,2019-04-01,01,Inbound,42976988.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01 05:29:00,1900-01-01 05:27:59,NaN,NaN
4,2019-04-01,01,Inbound,42977170.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01 05:30:00,1900-01-01 05:29:55,1200.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670643,2019-06-30,SL5,Outbound,44045095.0,15176,mawor,8.0,Midpoint,Headway,1900-01-02 00:54:00,NaN,900.0,NaN
7670644,2019-06-30,SL5,Outbound,44045095.0,55,Wasma,9.0,Midpoint,Headway,1900-01-02 00:55:00,NaN,900.0,NaN
7670645,2019-06-30,SL5,Outbound,44045095.0,60,Walen,10.0,Midpoint,Headway,1900-01-02 00:56:00,NaN,900.0,NaN
7670646,2019-06-30,SL5,Outbound,44045095.0,61,Melwa,11.0,Midpoint,Headway,1900-01-02 00:57:00,NaN,900.0,NaN


## Utility functions

In [21]:
def filter_on_row_col(df, target_rows, target_columns):
  for key, value in target_rows:
      df.drop(df[df[key] != value].index, inplace=True)
  df.drop(df.columns.difference(target_columns), axis=1, inplace=True)

def load_targets_from_csvs(target_columns, target_rows):
  bus_arrival_files = glob.glob('./*.csv')

  df_cumulative = pd.DataFrame()
  for f in bus_arrival_files:
    df_tmp = pd.read_csv(f)
    filter_on_row_col(df_tmp, target_rows, target_columns)
    df_cumulative = df_cumulative.append(df_tmp)
    print(f"Loaded {f}")
    del df_tmp

  return df_cumulative

# EDA on Routes Timeliness

In [22]:
df2 = load_targets_from_csvs(target_columns=['route_id','direction','stop_id'], target_rows=[])

Loaded .\MBTA Bus Arrival Departure Apr-June 2019.csv
Loaded .\MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded .\MBTA Bus Arrival Departure Jan-Mar 2019.csv
Loaded .\MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2019.csv


In [23]:
# Calculate visits per stop
group = df2[['route_id','direction','stop_id']].groupby(['route_id','direction'])
unique_stops = group.nunique().rename(columns={"stop_id":"unique_stop_ids"})
total_stops = pd.DataFrame(group.size()).rename(columns={0:"total_stops"})
stops = pd.merge(unique_stops, total_stops, left_index=True, right_index=True)
stops['visits_per_stop'] = stops['total_stops'] / stops['unique_stop_ids']

In [24]:
stops.sort_values('visits_per_stop', ascending=False).head(20)

unique_stop_ids  total_stops  visits_per_stop
route_id direction                                               
SL5      Outbound                12       814194     67849.500000
         Inbound                 13       872944     67149.538462
111      Inbound                 12       777438     64786.500000
SL1      Outbound                 7       448292     64041.714286
31       Outbound                 5       304750     60950.000000
         Inbound                  5       300727     60145.400000
SL2      Outbound                 6       345890     57648.333333
         Inbound                  6       335191     55865.166667
111      Outbound                13       718718     55286.000000
23       Inbound                 10       549635     54963.500000
SL1      Inbound                 13       709980     54613.846154
01       Outbound                 9       488393     54265.888889
         Inbound                  9       472691     52521.222222
39       Inbound                 11       561718     51065.272727
28       Inbound                 11       552235     50203.181818
39       Outbound                10       496903     49690.300000
743      Outbound                11       544599     49509.000000
77       Outbound                11       538967     48997.000000
         Inbound                 11       536599     48781.727273
23       Outbound                12       548555     45712.916667

# Preprocess data

Loading the data for a single route and a single direction

In [25]:
target_columns = ["service_date", "half_trip_id", "stop_id", "stop_sequence", "scheduled", "actual"]
target_rows = [('route_id', '01'),('direction', 'Inbound')]
df3 = load_targets_from_csvs(target_columns, target_rows)

Loaded .\MBTA Bus Arrival Departure Apr-June 2019.csv
Loaded .\MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded .\MBTA Bus Arrival Departure Jan-Mar 2019.csv
Loaded .\MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded .\MBTA Bus Arrival Departure Oct-Dec 2019.csv


Comment:
stop_id and stop_sequence are not always consistent in the dataframe!

In [26]:
group = df3.groupby(by=['stop_id', 'stop_sequence']).size()
print(group)

stop_id  stop_sequence
59       5.0                  1
         6.0                  3
         7.0              42083
62       1.0                  2
         6.0                  1
         7.0                  3
         8.0              42080
64       1.0                180
         2.0                  2
         7.0                  1
         8.0                  3
         9.0              42079
67       1.0                  3
         2.0              42069
72       1.0                  1
         2.0                  3
         3.0              42069
75       2.0                  1
         3.0                  3
         4.0              42083
79       3.0                  1
         4.0                  3
         5.0              42083
110      1.0              42070
187      4.0                  1
         5.0                  3
         6.0              42083
dtype: int64


Create a definition of valid stop IDs, based on the frequency of occurence

In [27]:
valid_stop_ids = group[group > group.max()/2]
sequence_to_id = valid_stop_ids.index.to_frame().reset_index(drop=True).sort_values(['stop_sequence']).set_index(['stop_id'])
print(sequence_to_id)
sequence_to_id = sequence_to_id['stop_sequence'].to_dict()

         stop_sequence
stop_id               
110                1.0
67                 2.0
72                 3.0
75                 4.0
79                 5.0
187                6.0
59                 7.0
62                 8.0
64                 9.0


Drop invalid (out of sequence) stop IDs

In [28]:
df3['expected_stop_sequence'] = df3['stop_id'].apply(lambda x: sequence_to_id[x])
df3.drop(df3[((df3['expected_stop_sequence'] != df3['stop_sequence']) | (df3['stop_sequence'].isna()))].index, axis=0, inplace=True)
df3.drop('expected_stop_sequence', axis=1, inplace=True)

Pivot data frame

In [33]:
df4 = df3.pivot(index=['service_date', 'half_trip_id'], columns=['stop_sequence'], values=['scheduled',	'actual'])

Drop incomplete routes

In [40]:
df5 = df4[df4.isna().sum(axis=1) == 0]

## Transform data

In [41]:
# Transform to datetime
time_format = '%Y-%m-%d %H:%M:%S'
df5 = df5.applymap(lambda x: datetime.datetime.strptime(x, time_format))
df5

scheduled                      \
stop_sequence                             1.0                 2.0   
service_date half_trip_id                                           
2018-08-02   40135101.0   1900-01-01 05:10:00 1900-01-01 05:13:00   
             40135103.0   1900-01-01 06:21:00 1900-01-01 06:24:00   
             40135105.0   1900-01-01 07:44:00 1900-01-01 07:49:00   
             40135121.0   1900-01-01 07:06:00 1900-01-01 07:10:00   
             40135155.0   1900-01-01 06:06:00 1900-01-01 06:09:00   
...                                       ...                 ...   
2019-09-30   45106583.0   1900-01-01 16:24:00 1900-01-01 16:29:00   
             45106585.0   1900-01-01 18:33:00 1900-01-01 18:38:00   
             45106587.0   1900-01-01 20:11:00 1900-01-01 20:15:00   
             45106589.0   1900-01-01 21:33:00 1900-01-01 21:37:00   
             45108654.0   1900-01-01 18:18:00 1900-01-01 18:23:00   

                                                                   \
stop_sequence                             3.0                 4.0   
service_date half_trip_id                                           
2018-08-02   40135101.0   1900-01-01 05:16:00 1900-01-01 05:19:00   
             40135103.0   1900-01-01 06:27:00 1900-01-01 06:30:00   
             40135105.0   1900-01-01 07:55:00 1900-01-01 08:00:00   
             40135121.0   1900-01-01 07:13:00 1900-01-01 07:17:00   
             40135155.0   1900-01-01 06:12:00 1900-01-01 06:15:00   
...                                       ...                 ...   
2019-09-30   45106583.0   1900-01-01 16:35:00 1900-01-01 16:42:00   
             45106585.0   1900-01-01 18:44:00 1900-01-01 18:49:00   
             45106587.0   1900-01-01 20:19:00 1900-01-01 20:24:00   
             45106589.0   1900-01-01 21:41:00 1900-01-01 21:46:00   
             45108654.0   1900-01-01 18:29:00 1900-01-01 18:36:00   

                                                                   \
stop_sequence                             5.0                 6.0   
service_date half_trip_id                                           
2018-08-02   40135101.0   1900-01-01 05:22:00 1900-01-01 05:25:00   
             40135103.0   1900-01-01 06:35:00 1900-01-01 06:39:00   
             40135105.0   1900-01-01 08:05:00 1900-01-01 08:11:00   
             40135121.0   1900-01-01 07:22:00 1900-01-01 07:26:00   
             40135155.0   1900-01-01 06:18:00 1900-01-01 06:21:00   
...                                       ...                 ...   
2019-09-30   45106583.0   1900-01-01 16:51:00 1900-01-01 16:59:00   
             45106585.0   1900-01-01 18:56:00 1900-01-01 19:02:00   
             45106587.0   1900-01-01 20:29:00 1900-01-01 20:34:00   
             45106589.0   1900-01-01 21:51:00 1900-01-01 21:56:00   
             45108654.0   1900-01-01 18:45:00 1900-01-01 18:51:00   

                                                                   \
stop_sequence                             7.0                 8.0   
service_date half_trip_id                                           
2018-08-02   40135101.0   1900-01-01 05:28:00 1900-01-01 05:32:00   
             40135103.0   1900-01-01 06:42:00 1900-01-01 06:49:00   
             40135105.0   1900-01-01 08:16:00 1900-01-01 08:24:00   
             40135121.0   1900-01-01 07:29:00 1900-01-01 07:37:00   
             40135155.0   1900-01-01 06:24:00 1900-01-01 06:28:00   
...                                       ...                 ...   
2019-09-30   45106583.0   1900-01-01 17:05:00 1900-01-01 17:13:00   
             45106585.0   1900-01-01 19:07:00 1900-01-01 19:12:00   
             45106587.0   1900-01-01 20:38:00 1900-01-01 20:42:00   
             45106589.0   1900-01-01 22:00:00 1900-01-01 22:04:00   
             45108654.0   1900-01-01 18:56:00 1900-01-01 19:01:00   

                                                           actual  \
stop_sequence                             9.0                 1.0   
service_date half_trip_id     

# Training

## TODO

# Inference

## TODO